In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# Load joined finance + vendor data
df_finance = spark.read.format("delta").load("/mnt/delta/silver/finance_with_vendor_info")

# Load Silver-level cleaned vendor registry
df_registry = spark.read.format("delta").load("/mnt/delta/silver/vendor_registry_clean")

# Join on vendor_id
df_enriched = df_finance.join(df_registry, on="vendor_id", how="left")

# Write final Gold table
df_enriched.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save("/mnt/delta/silver/finance_with_vendors_enriched")

# Register as SQL table
spark.sql("DROP TABLE IF EXISTS finance_with_vendors_enriched")
spark.sql("""
CREATE TABLE finance_with_vendors_enriched
USING DELTA
LOCATION '/mnt/delta/silver/finance_with_vendors_enriched'
""")